In [1]:
import pandas as pd
import numpy as np
import datetime
import boto3
from matplotlib import pyplot as plt
from matplotlib.dates import DateFormatter, date2num
import seaborn as sns
import scipy.stats as stats

In [2]:
#reads in 10k row sample of Kiva's loan data
loan = pd.read_csv('../data/sample_data.csv')

In [3]:
#converting string representations of time to Datetime objects
loan['posted_datetime'] = pd.to_datetime(loan['POSTED_TIME'])
loan['raised_datetime'] = pd.to_datetime(loan['RAISED_TIME'])

#creates loan_speed column as difference between raised and posted datetimes
loan['loan_speed'] = loan['raised_datetime']-loan['posted_datetime']

#represents time to raising loan in number of days (for matplotlib)
loan['loanspeed_days'] = loan['loan_speed'] / pd.Timedelta(hours=24)

#creates loan_year column 
loan['loan_year'] = [loan['raised_datetime'][x].year for x in loan.index]


In [8]:
#Function for counting number of borrowers from number of entries in gender column 
def count_borrowers(lst):
    if type(lst) != float:
        return len(lst.split(','))
    else:
        return 1

loan['borrower_n'] = loan['BORROWER_GENDERS'].apply(count_borrowers)


In [22]:
#reads in csv of purchasing power parity values from world bank
#first four rows are skipped to avoid irrelevant metadata
ppp = pd.read_csv('../data/world_bank_ppp.csv', skiprows=4)

#Renames Country Name column to match loan dataframe for merge 
ppp.rename(columns={'Country Name': 'COUNTRY_NAME'}, inplace=True)

In [27]:
#merging ppp data for years where Kiva loans were active (2006-2019)
loan = loan.merge(how='inner', on='COUNTRY_NAME', 
                  right=ppp[['COUNTRY_NAME', '2006', '2007', '2008', '2009', '2010', '2011',
                            '2012', '2013', '2014', '2015', '2016','2017', '2018', '2019']])


In [28]:
loan.columns

Index(['Unnamed: 0', 'LOAN_ID', 'LOAN_NAME', 'ORIGINAL_LANGUAGE',
       'DESCRIPTION', 'DESCRIPTION_TRANSLATED', 'FUNDED_AMOUNT', 'LOAN_AMOUNT',
       'STATUS', 'IMAGE_ID', 'VIDEO_ID', 'ACTIVITY_NAME', 'SECTOR_NAME',
       'LOAN_USE', 'COUNTRY_CODE', 'COUNTRY_NAME', 'TOWN_NAME',
       'CURRENCY_POLICY', 'CURRENCY_EXCHANGE_COVERAGE_RATE', 'CURRENCY',
       'PARTNER_ID', 'POSTED_TIME', 'PLANNED_EXPIRATION_TIME', 'DISBURSE_TIME',
       'RAISED_TIME', 'LENDER_TERM', 'NUM_LENDERS_TOTAL',
       'NUM_JOURNAL_ENTRIES', 'NUM_BULK_ENTRIES', 'TAGS', 'BORROWER_NAMES',
       'BORROWER_GENDERS', 'BORROWER_PICTURED', 'REPAYMENT_INTERVAL',
       'DISTRIBUTION_MODEL', 'posted_datetime', 'raised_datetime',
       'loan_speed', 'loanspeed_days', 'loan_year', 'borrower_n',
       'Country Name', '2006_x', '2007_x', '2008_x', '2009_x', '2010_x',
       '2011_x', '2012_x', '2013_x', '2014_x', '2015_x', '2016_x', '2017_x',
       '2018_x', '2019_x', '2006_y', '2007_y', '2008_y', '2009_y', '2010_y',
